<a href="https://colab.research.google.com/github/Pankaj122002/Python/blob/main/Text_Analysis(Web_scraping).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [67]:
!pip install requests
!pip install beautifulsoup4
!pip install textblob
!pip install syllables

In [68]:
import requests
from bs4 import BeautifulSoup
from textblob import TextBlob
import pandas as pd
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [69]:
def extract_article_text(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')

    # Find and extract the article title and text
    article_title = soup.find('title').get_text()
    article_text = ""
    paragraphs = soup.find_all(['p', 'div'])  # Adjust based on the actual structure
    for p in paragraphs:
        article_text += p.get_text()

    return article_title, article_text

In [70]:
input_df = pd.read_excel('/content/Input.xlsx')

# Initialize a list to store analysis results
results = []

# Iterate through each row in the input DataFrame
for index, row in input_df.iterrows():
    url_id = row['URL_ID']
    url = row['URL']


In [71]:
article_title, article_text = extract_article_text(url)



In [72]:
# Perform text analysis using TextBlob
b = TextBlob(article_text)



In [73]:
positive_score = b.sentiment.polarity
negative_score = b.sentiment.polarity
polarity_score = b.sentiment.polarity
subjectivity_score = b.sentiment.subjectivity
avg_sentence_length = len(b.words) / len(b.sentences)


In [95]:
import re
def count_syllables(word):
    # Convert the word to lowercase
    word = word.lower()

    # Define a regular expression to match consecutive vowels
    # but not ending with 'e'
    pattern = '(?!e$)[aeiouy]+'

    # Find all matches of the pattern in the word
    matches = re.findall(pattern, word, re.I)

    # Return the count of matches (syllables)
    return len(matches)

# Example usage
word = "syllable"
syllable_count = count_syllables(word)

In [106]:
def count_complex_words(b):
    complex_word_count = sum(1 for word in b.words if syllable_count >= 3)
    return complex_word_count




In [111]:
complex_word_count = count_complex_words(b)

In [110]:
percentage_complex_words = (complex_word_count / len(b.words)) * 100



In [112]:
fog_index = 0.4 * (avg_sentence_length + percentage_complex_words)


In [113]:
avg_words_per_sentence = len(b.words) / len(b.sentences)


In [114]:
word_count = len(b.words)

In [116]:
avg_syllables_per_word = syllable_count / word_count

In [118]:
personal_pronoun_count = sum(1 for word in b.words if word.lower() in ['i', 'you', 'he', 'she', 'we', 'they'])

In [119]:
avg_word_length = sum(len(word) for word in b.words) / word_count

In [75]:
results = []

In [120]:
result = {
        'URL_ID': url_id,
        'POSITIVE SCORE': positive_score,
        'NEGATIVE SCORE': negative_score,
        'POLARITY SCORE': polarity_score,
        'SUBJECTIVITY SCORE': subjectivity_score,
        'AVG SENTENCE LENGTH': avg_sentence_length,
        'PERCENTAGE OF COMPLEX WORDS': percentage_complex_words,
        'FOG INDEX':fog_index,
        'AVG NUMBER OF WORDS PER SENTENCE' :avg_words_per_sentence,
        'COMPLEX WORD COUNT': complex_word_count,
        'WORD COUNT': word_count,
        'SYLLABLE PER WORD': avg_syllables_per_word,
        'PERSONAL PRONOUNS': personal_pronoun_count,
        'AVG WORD LENGTH': avg_word_length
    }
results.append(result)

In [121]:
output_df = pd.DataFrame(results)

In [123]:
output_df.to_excel('output.xlsx', index=False)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# New Section